In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

In [ ]:
# Create Vine Data Base
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3PMQXEVYR4ZT1|          3|            0|          0|   N|                Y|
|R1I1K3EO98EMUV|          5|            0|          0|   N|                Y|
|R3K23URSI2MY42|          5|            4|          4|   N|                Y|
| RH2P770T10X38|          5|            0|          0|   N|                Y|
| RV9YBAW8CWFYN|          1|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# Filtering to total votes >20
filtered_vine_df=vine_df.filter("total_votes>20")
filtered_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1Z9QZRQJ1F8JX|          1|           17|         22|   N|                Y|
|R31PAYBVBP8W62|          1|           54|         81|   N|                Y|
|R33GCMAT1K5DX8|          2|          141|        190|   N|                Y|
|R3MIL83RY4SMS4|          1|           27|         38|   N|                Y|
|R37T9FXGG07XOM|          1|           24|         28|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# Filtering helpful votes % > 50%
hvotes_vine_df = filtered_vine_df.filter(filtered_vine_df.helpful_votes/filtered_vine_df.total_votes>=0.5)
hvotes_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1Z9QZRQJ1F8JX|          1|           17|         22|   N|                Y|
|R31PAYBVBP8W62|          1|           54|         81|   N|                Y|
|R33GCMAT1K5DX8|          2|          141|        190|   N|                Y|
|R3MIL83RY4SMS4|          1|           27|         38|   N|                Y|
|R37T9FXGG07XOM|          1|           24|         28|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [21]:
# Filtering Paid ones

paid_vine_df = hvotes_vine_df.filter("vine == 'Y'")
paid_vine_df.show(5)

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [20]:
# Filtering No Paid ones

no_paid_vine_df = hvotes_vine_df.filter("vine == 'N'")
no_paid_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1Z9QZRQJ1F8JX|          1|           17|         22|   N|                Y|
|R31PAYBVBP8W62|          1|           54|         81|   N|                Y|
|R33GCMAT1K5DX8|          2|          141|        190|   N|                Y|
|R3MIL83RY4SMS4|          1|           27|         38|   N|                Y|
|R37T9FXGG07XOM|          1|           24|         28|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [33]:
#Paid Summary Stats

paid_summary=[paid_vine_df.count(),paid_vine_df.filter("star_rating == 5").count(), 0]


In [35]:
#No Paid Summary Stats
no_paid_summary=[no_paid_vine_df.count(),no_paid_vine_df.filter("star_rating == 5").count(),no_paid_vine_df.filter("star_rating==5").count()*100/no_paid_vine_df.count()]

In [41]:
#Create Summary

import pandas as pd
Summary = pd.DataFrame({
    "Paid":paid_summary,
    "No Paid":no_paid_summary
},
index=["Total Reviews", "5 Star Reviews", "%"])
Summary

,Paid,No Paid
Total Reviews,0,123420.00000
5 Star Reviews,0,56739.00000
%,0,45.97229
